<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/merge/fire_clip_to_roi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clip Fire Data to Algeria + Tunisia

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 94.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_path = '/content/drive/MyDrive/dm_fire_prediction/fire_keep_all_fires_approach.csv'
fire_df = pd.read_csv(input_path)

In [ ]:
fire_df.head()

,Unnamed: 0,latitude,longitude,fire
0,0,35.70751,5.53337,1
1,1,32.13579,6.46961,1
2,2,32.35563,6.97630,1
3,3,28.19791,9.39581,1
4,4,28.12826,9.49323,1


In [ ]:
fire_df = fire_df[['longitude', 'latitude', 'fire']]
fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2863369 entries, 0 to 2863368
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   longitude  float64
 1   latitude   float64
 2   fire       int64  
dtypes: float64(2), int64(1)
memory usage: 65.5 MB


In [ ]:
### Loading and merging the boundaries of Algeria and Tunisia
def load_dz_tn_boundaries(dz_shapefile_path, tn_shapefile_path):
    dz = gpd.read_file(dz_shapefile_path)
    tn = gpd.read_file(tn_shapefile_path)

    # Ensure both shapefiles use the same CRS
    if dz.crs != tn.crs:
        tn = tn.to_crs(dz.crs)

    # Concatenate the GeoDataFrames
    boundaries = gpd.GeoDataFrame(pd.concat([dz, tn], ignore_index=True), crs=dz.crs)

    return boundaries



# def clip_instance(file, boundaries=boundaries):
#     with rasterio.open(file) as src:
#         # Clip raster with shapefile
#         out_image, out_transform = mask(src, boundaries.geometry, crop=True)
#         out_meta = src.meta.copy()

#     # Update metadata
#     out_meta.update({
#         "driver": "GTiff",
#         "height": out_image.shape[1],
#         "width": out_image.shape[2],
#         "transform": out_transform
#     })

#     return out_image, out_meta

In [ ]:
dz_boundaries_filepath = '/content/drive/MyDrive/dm_fire_prediction/borders/dz_shp'
tn_boundaries_filepath = '/content/drive/MyDrive/dm_fire_prediction/borders/tn_shp'

boundaries = load_dz_tn_boundaries(dz_boundaries_filepath, tn_boundaries_filepath)
boundaries

/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'dz_shp': 'gadm41_DZA_2' (default), 'gadm41_DZA_1', 'gadm41_DZA_0'. Specify layer parameter to avoid this warning.
  result = read_func(
/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'tn_shp': 'gadm41_TUN_2' (default), 'gadm41_TUN_1', 'gadm41_TUN_0'. Specify layer parameter to avoid this warning.
  result = read_func(


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,DZA.1.1_1,DZA,Algeria,DZA.1_1,Adrar,ولاية أدرا,Adrar,Adrār,NA,Chef-Lieu-Wilaya,Chef-Lieu-Wilaya,101,DZ.AR.AD,"POLYGON ((-0.14846 27.76886, -0.15202 27.77545..."
1,DZA.1.2_1,DZA,Algeria,DZA.1_1,Adrar,ولاية أدرا,Akabli,Akabili,NA,Commune,Commune,119,DZ.AR.AK,"POLYGON ((0.9337 26.23569, 0.90476 26.29767, 1..."
2,DZA.1.3_1,DZA,Algeria,DZA.1_1,Adrar,ولاية أدرا,Aougrout,NA,NA,Commune,Commune,123,DZ.AR.AG,"POLYGON ((2.04078 28.89772, 2.04051 28.89059, ..."
3,DZA.1.4_1,DZA,Algeria,DZA.1_1,Adrar,ولاية أدرا,Aoulef,Awla-f,NA,Commune,Commune,112,DZ.AR.AL,"POLYGON ((1.28069 26.78899, 1.13273 26.74567, ..."
4,DZA.1.5_1,DZA,Algeria,DZA.1_1,Adrar,ولاية أدرا,Bordj Badji Mokhtar,NA,NA,Commune,Commune,125,DZ.AR.BB,"POLYGON ((1.42782 23.99674, 1.42931 23.95749, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,TUN.24.2_1,TUN,Tunisia,TUN.24_1,Zaghouan,NA,Fahs,NA,NA,Delegation,Delegation,NA,TN.ZA.FA,"POLYGON ((9.84352 36.09752, 9.83557 36.09973, ..."
1768,TUN.24.3_1,TUN,Tunisia,TUN.24_1,Zaghouan,NA,Nadhour,NA,NA,Delegation,Delegation,NA,TN.ZA.NA,"POLYGON ((9.85318 36.09475, 9.84802 36.09626, ..."
1769,TUN.24.4_1,TUN,Tunisia,TUN.24_1,Zaghouan,NA,Saouaf,NA,NA,Delegation,Delegation,NA,TN.ZA.SA,"POLYGON ((10.19923 36.16115, 10.1973 36.16191,..."
1770,TUN.24.5_1,TUN,Tunisia,TUN.24_1,Zaghouan,NA,Zaghouan,NA,NA,Delegation,Delegation,NA,TN.ZA.ZA,"POLYGON ((10.23154 36.47617, 10.23126 36.47531..."


In [ ]:
def clip(fire_df, boundaries):
    # Convert fire_df to a GeoDataFrame
    fire_gdf = gpd.GeoDataFrame(
        fire_df,
        geometry=gpd.points_from_xy(fire_df.longitude, fire_df.latitude),
        crs="EPSG:4326" # WGS84, common CRS for lat/lon
    )

    # Ensure the fire_gdf has the same CRS as the boundaries
    if fire_gdf.crs != boundaries.crs:
        fire_gdf = fire_gdf.to_crs(boundaries.crs)

    # Perform a spatial join to keep only the points within the boundaries
    # 'inner' join means only points that intersect with a polygon in boundaries will be kept
    clipped_fire_gdf = gpd.sjoin(fire_gdf, boundaries, how="inner", predicate='intersects')

    # Drop the 'index_right' column which is added by sjoin
    clipped_fire_gdf = clipped_fire_gdf.drop(columns=['index_right'])

    # Convert back to a pandas DataFrame if needed, or return GeoDataFrame
    # For this task, let's return a pandas DataFrame for consistency with input fire_df
    return pd.DataFrame(clipped_fire_gdf.drop(columns='geometry'))

In [ ]:
clipped_fire_df = clip(fire_df, boundaries)
clipped_fire_df.head()

,longitude,latitude,fire,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2
0,5.53337,35.70751,1,DZA.6.24_1,DZA,Algeria,DZA.6_1,Batna,ولاية باتنة,Guiga,Guigba,NA,Commune,Commune,510,DZ.BT.GU
1,6.46961,32.13579,1,DZA.33.8_1,DZA,Algeria,DZA.33_1,Ouargla,ورقلة,Hassi Messaoud,Hāssī Mas`ūd,NA,Commune,Commune,3004,DZ.OG.HM
2,6.97630,32.35563,1,DZA.18.5_1,DZA,Algeria,DZA.18_1,El Oued,ولاية الوادي,Douar El Ma,NA,NA,Commune,Commune,3915,DZ.EO.DM
3,9.39581,28.19791,1,DZA.22.6_1,DZA,Algeria,DZA.22_1,Illizi,ولاية اليزي,In Amenas,NA,NA,Commune,Commune,3306,DZ.IL.IA
4,9.49323,28.12826,1,DZA.22.6_1,DZA,Algeria,DZA.22_1,Illizi,ولاية اليزي,In Amenas,NA,NA,Commune,Commune,3306,DZ.IL.IA


In [ ]:
clipped_fire_df.reset_index(drop=True, inplace=True)
clipped_fire_df = clipped_fire_df[['longitude', 'latitude', 'fire']]
clipped_fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1965256 entries, 0 to 1965255
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   longitude  float64
 1   latitude   float64
 2   fire       int64  
dtypes: float64(2), int64(1)
memory usage: 45.0 MB


In [ ]:
output_path = '/content/drive/MyDrive/dm_fire_prediction/premerge/fire.csv'

clipped_fire_df.to_csv(output_path, index=False)